# Building AI for BlackJack Game

This notebook is for explorartion purposes. I keep this notebook as a journal.

- This idea has been started with a conversation that I have made with my nephew. He asked me if we could build an AI agent that plays the best moves for blackjack. After some arguing on the topic, we have agreed that it would be overkill to train a deep net. Hence there were already best posible moves avaible. However this table only keeps the best action to play in a total matrix of player and dealer. So we have agreed to at least to construct a q-value table via reinforcement learning. We are expecting to construct a q-value table that will consists of the best move to to make (Hit,Double,Stand) in percentages. Our intension is to keep it simple so we will not implement the full rrules of the game. We will consider Ace as 1 not 11. I've learned that this is called hard totals. The avaible best moves are as follows:

![Best Action Table](best_moves_in_hard_totals.jpeg)

- So our plan is to put the percentages of each action for each state. (Q-Values)
- We are also aware of [this](https://towardsdatascience.com/reinforcement-learning-solving-blackjack-5e31a7fb371f) page. We may check it out in the future. But for learning purposes I will check it out if I geet stuck of my own.

28.05.2020

- I start right away coding the environment, I need to remember my other projects.

In [ ]:
class environment:
    